In [98]:
# we want to calculate Berkshire Hathaway Class B(BRK-B) stock's value one year from today 

In [99]:
# given_data is a dictionary of stock to a tuple of information
given_data = {}

In [100]:
# apple share may 2020 = $300.63, av share price 2019 was = $ 280.10, and delta is $20.53
given_data["APPL"] = (300.63, 280.10, 20.53)
print(given_data["APPL"][0])

300.63


In [101]:
# add rest of derived data
given_data["BAC"] = (22.35, 29.89, -7.54)
given_data["KO"] = (44.75, 52.24, -7.79)
given_data["AXP"] = (20.12, 25.62, -5.50)
given_data["WFC"] = (25.61, 43.97, -18.36)
print(given_data["AXP"][0])

20.12


In [102]:
# derived_price will store share prices of stocks prices in may 2021
derived_price = {}

In [103]:
# calc price of stock share by using formula A + -k(dA) where A is price today, and dA is price change from last yr
def calcPrice(given_data):
    for stock,data in given_data.items(): 
        price_now = data[0]
        price_av2019 = data[1]
        delta = price_now - price_av2019
        k = -(price_now / price_av2019)
        derived_price[stock] = round(price_now + k*delta,2)

In [104]:
calcPrice(given_data)
derived_price

{'APPL': 278.6, 'BAC': 27.99, 'KO': 51.17, 'AXP': 24.44, 'WFC': 36.3}

In [105]:
# Awesome! it worked... we have an esitmate of price may 2021 for the holdings... to see them agian
print(derived_price)

{'APPL': 278.6, 'BAC': 27.99, 'KO': 51.17, 'AXP': 24.44, 'WFC': 36.3}


In [106]:
# lets add what percentage of Berkshire Hathaway's revune is from each of the stocks' divendeds 
dividend_equity = {}

In [107]:
# APPL was 19.6% of Berkshire Hathaway's price. So...
dividend_equity["APPL"] = 19.6
print(dividend_equity["APPL"])

19.6


In [108]:
# Add remainng equities 
dividend_equity["BAC"] = 14.5
dividend_equity["KO"] = 16.4
dividend_equity["AXP"] = 6.2
dividend_equity["WFC"] = 21.3
print(dividend_equity["WFC"])

21.3


In [109]:
# find sum of dividened stakes
def calcTotalEquity(dividend_equity):
    sum = 0
    for percentage in dividend_equity.values():
        sum += percentage
    return sum
calcTotalEquity(dividend_equity)

78.0

In [110]:
# add remaining stakes...did this if we wanted to add another Berkshire Hathaway stock later...code is modular + ready
dividend_equity["OTHER_STAKES"] = 100 - calcTotalEquity(dividend_equity)
# total equity now should be 100
calcTotalEquity(dividend_equity)

100.0

In [111]:
# # get other stakes percentage...78 percent make the above comapnies...we want to represent the remaining as its own stock
# OTS_percent = dividend_equity["OTHER_STAKES"] /100
given_data["OTHER_STAKES"] = (172.38, 210.55, -38.17)
calcPrice(given_data)
print(derived_price)

{'APPL': 278.6, 'BAC': 27.99, 'KO': 51.17, 'AXP': 24.44, 'WFC': 36.3, 'OTHER_STAKES': 203.63}


In [112]:
# calc berk-B new price in may 2021
def calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price):
    newPrice = 0
    for stock, price in derived_price.items():
        newPrice += price*dividend_equity[stock]/100
    updatedProportion = round(brk_price * newPrice /100, 2)
    print("BRK-B price day vaccine found $",brk_price, "\nBRK-B price in May 2021 $", updatedProportion)
    return updatedProportion

brk_price = given_data["OTHER_STAKES"][0]
calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price)

BRK-B price day vaccine found $ 172.38 
BRK-B price in May 2021 $ 208.76


208.76

In [113]:
# lets try to remodel our program if a vaccine was found 30 days from now instead of today
# to do this...twe assume the stocks that were falling due to covid 19 will keep falling...vice versa
# if BAC shares are down to 22.35 from 29.89 in 2019...they will keep decling till vaccine is found
# the decline formula we will use is a = a + x*(dA/365) 
# where x is the days from today a vaccine is found dA is a - av2019
# dBAC = -7.54 so if vccine was found 30 days from today we predict BAC will be = 22.35 + 30(-7.54/365) = 21.73
# for stocks with dA < -7.54 like -30 will drop faster than BA

In [114]:
# store updated stock prices if vaccine was found 30 days from today
# pastData is of form Key:Value where key is the ticker and value is tuple (stockToday, stock2019, change)
data_in_30days = {}
def calcUpdatedData(pastData, newData, numOfDays):
    for key, value in pastData.items():
        oldPrice = value[0]
        price_av2019 = value[1]
        newPrice = updateSharePrice(oldPrice, oldPrice - price_av2019, numOfDays)
        newData[key] = (newPrice, price_av2019)
        
def updateSharePrice(oldPrice, delta, days):
    return round(oldPrice + days*delta/365,2)

In [115]:
calcUpdatedData(given_data, data_in_30days, 30)
print(given_data, data_in_30days)

{'APPL': (300.63, 280.1, 20.53), 'BAC': (22.35, 29.89, -7.54), 'KO': (44.75, 52.24, -7.79), 'AXP': (20.12, 25.62, -5.5), 'WFC': (25.61, 43.97, -18.36), 'OTHER_STAKES': (172.38, 210.55, -38.17)} {'APPL': (302.32, 280.1), 'BAC': (21.73, 29.89), 'KO': (44.13, 52.24), 'AXP': (19.67, 25.62), 'WFC': (24.1, 43.97), 'OTHER_STAKES': (169.24, 210.55)}


In [121]:
# a better way of visualizing the change in data
def compareData(pastData, newData, numOfDays):
    print("heres the changes in prices if vaccine was found in " + str(numOfDays) +" days...\n")
    for key, value in pastData.items():
        print(key,"", value[0], "-->", newData[key][0])
compareData(given_data, data_in_30days, 30)

heres the changes in prices if vaccine was found in 30 days...

APPL  300.63 --> 302.32
BAC  22.35 --> 21.73
KO  44.75 --> 44.13
AXP  20.12 --> 19.67
WFC  25.61 --> 24.1
OTHER_STAKES  172.38 --> 169.24


In [117]:
# lets store the calcualted price in an another dict with values of arrays of prices based on what day vaccince was found
data_in_Xdays = {}
for key in given_data:
    data_in_Xdays[key] = []
print(data_in_Xdays)

def storeDataInXDays(data_in_Xdays, given_data): 
    for key in given_data:
        data_in_Xdays[key].append(given_data[key][0])
        
# storing data of individual stock prices if vaccine was found today 
storeDataInXDays(data_in_Xdays, given_data)
print(data_in_Xdays)

# storing data of individual stock prices if vaccine was found 30 days from today
storeDataInXDays(data_in_Xdays, data_in_30days)
print(data_in_Xdays)

{'APPL': [], 'BAC': [], 'KO': [], 'AXP': [], 'WFC': [], 'OTHER_STAKES': []}
{'APPL': [300.63], 'BAC': [22.35], 'KO': [44.75], 'AXP': [20.12], 'WFC': [25.61], 'OTHER_STAKES': [172.38]}
{'APPL': [300.63, 302.32], 'BAC': [22.35, 21.73], 'KO': [44.75, 44.13], 'AXP': [20.12, 19.67], 'WFC': [25.61, 24.1], 'OTHER_STAKES': [172.38, 169.24]}


In [118]:
# lets calc our BRK price again if a vaccine was found today 
print("if vaccine was found today...")
brk_price = given_data["OTHER_STAKES"][0]
futurePrice = calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price)

brk_prices_if_vaccineXdays = []
brk_prices_if_vaccineXdays.append((0,futurePrice))

if vaccine was found today...
BRK-B price day vaccine found $ 172.38 
BRK-B price in May 2021 $ 208.76


In [119]:
# now clac updatedBRK Price if Vaccine found in 30 days
print("if vaccine was found 30 days from today...")
calcPrice(data_in_30days)
brk_price = data_in_30days["OTHER_STAKES"][0]
futurePrice = calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price)

brk_prices_if_vaccineXdays = []
brk_prices_if_vaccineXdays.append((0,futurePrice))

if vaccine was found 30 days from today...
BRK-B price day vaccine found $ 169.24 
BRK-B price in May 2021 $ 203.8


## Results are positive! 🎉
If vaccine was found later from today BRK shares would be a little lower than if it was found today...as data shows

lets test our modular code and put it to the test by determining BRK price if vaccine was found in X days...we will call:

- calcUpdatedData(pastData, newData, numOfDays)
- compareData(pastData, newData, numOfDays)
- storeDataInXDays(data_in_Xdays, newData)
- calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price)
- brk_prices_if_vaccineXdays.append((0,futurePrice))

**This is our update function**

In [125]:
def vaccineFoundInXDays(numOfDays):
    data_before_vaccine_found = {}
    calcUpdatedData(given_data, data_before_vaccine_found , numOfDays)
    compareData(given_data, data_before_vaccine_found , numOfDays)
    storeDataInXDays(data_in_Xdays, data_before_vaccine_found )
    
    calcPrice(data_before_vaccine_found )
    brk_price = data_before_vaccine_found["OTHER_STAKES"][0]
    print("\n")
    
    futurePrice = calcUpdatedBRKPrice(derived_price, dividend_equity, brk_price)
    brk_prices_if_vaccineXdays = []
    brk_prices_if_vaccineXdays.append((numOfDays,futurePrice))


lets call our update function with if we found vaccine in 30 days and see if matches with the previous results...
> if vaccine was found 30 days from today... 

> BRK-B price day vaccine found $ 169.24 

> BRK-B price in May 2021 $ 203.8

In [123]:
vaccineFoundInXDays(30)

heres the changes in prices if vaccine was found in 30 days...

APPL  300.63 --> 302.32
BAC  22.35 --> 21.73
KO  44.75 --> 44.13
AXP  20.12 --> 19.67
WFC  25.61 --> 24.1
OTHER_STAKES  172.38 --> 169.24
BRK-B price day vaccine found $ 169.24 
BRK-B price in May 2021 $ 203.8


## Results are positive! 🎉
results from update function AKA vaccineFoundInXDays(numOFDays) match the results calculated prev


Lets calculate BRK-B Prices for 60 days and 90 days...
each should be getting smaller as the days a vaccine found increaeses

In [126]:
vaccineFoundInXDays(60)

heres the changes in prices if vaccine was found in 60 days...

APPL  300.63 --> 304.0
BAC  22.35 --> 21.11
KO  44.75 --> 43.52
AXP  20.12 --> 19.22
WFC  25.61 --> 22.59
OTHER_STAKES  172.38 --> 166.11
/n
BRK-B price day vaccine found $ 166.11 
BRK-B price in May 2021 $ 198.81
